In [1]:
import pandas as pd
import numpy as np 
import datetime
import ast
from waste_functions import predicted_demand, waste_analysis
inventory = pd.read_csv("data/inventory.csv")
products = pd.read_csv("data/products.csv")
promotions = pd.read_csv("data/promotions.csv")
transactions = pd.read_csv("data/transactions.csv")
discount_per_day = pd.read_csv("nr_discounts_per_day.csv")
pd.set_option('mode.chained_assignment', None) #turns off pandas warnings
#create a reference dataframe
dayofyear = list(range(365))
date = [datetime.datetime(2018, 1, 1) + datetime.timedelta(day) for day in dayofyear]
df_date = pd.DataFrame({"day":dayofyear, "date":date})

tup_list2 = [('Blauwe bessen', '150g'), ('Rundergehakt', '300g '), ('Unox Gelderse rookworst', '285g'), ('Biologisch rundergehakt', '300g')]
for pair in tup_list2:

    transactions.loc[transactions[(transactions['description'] == pair[0]) & (transactions['size'] == pair[1])].index, 'description'] = pair[0] + '.1'

    products.loc[products[(products['description'] == pair[0]) & (products['size'] == pair[1])].index, 'description'] = pair[0] + '.1'

#for colname in inventory.columns[2:]:
#    inventory[colname] = [ast.literal_eval(i) for i in inventory[colname]]
    #converting inventory rows to list of tuples from string

product = "Eierkoeken" # item to be analyzed
filtered_transactions = transactions[transactions["description"] == product]
filtered_inventory = inventory[["day", "before or after delivery", product]]

## Testing `waste analysis` function 

In [9]:
complimentary = "Biologisch volkoren heel"
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
rf = RandomForestRegressor(n_estimators=50)
dtc = DecisionTreeRegressor(criterion="mae")
output_full = waste_analysis(inventory, transactions, products, product, discount_per_day=discount_per_day)

output_full

(76, 7)
Extended waste dataframe created
Data prepared for prediction
Model fitted; test set metrics: MAE: 5.3112624680954905 MSE: 43.26675861223043, R^2 score: 0.3341249548189359
Demand function fitted
Average of actual/estimated purchase ratios: 1.0053999221263994
Waste change predicted, calculating revenue loss...


,amount,total_inventory,remaining_stock,DOY,cumulative purchases,purchases,remaining,waste,week,waste nn,...,avg_price,std_price,expected purchases,expected waste,expected loss revenue,expected loss profit,expected waste cost,loss revenue,loss profit,waste cost
best before day,,,,,,,,,,,,,,,,,,,,,
100,125,None,None,100.0,3338.0,NaN,-3213.0,-3213.0,15,0,...,1.640139,1.684029,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0
103,125,None,None,103.0,3430.0,92.0,33.0,-3180.0,15,0,...,1.657951,1.790000,98.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
107,125,None,None,107.0,3567.0,137.0,-12.0,-3192.0,16,0,...,1.637036,1.790000,146.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
110,125,None,None,110.0,3659.0,92.0,33.0,-3159.0,16,0,...,1.757572,1.790000,98.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
114,125,None,None,114.0,3778.0,119.0,6.0,-3153.0,17,0,...,1.747441,1.790000,127.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
348,150,None,None,348.0,11053.0,93.0,57.0,-1803.0,50,0,...,1.667104,1.790000,99.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
352,150,None,None,352.0,11184.0,131.0,19.0,-1784.0,51,0,...,1.665908,1.790000,139.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
355,150,None,None,355.0,11281.0,97.0,53.0,-1731.0,51,0,...,1.654394,1.790000,103.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [2]:
money_cols = ["loss revenue", "loss profit", "waste cost", "expected loss revenue", "expected loss profit", "expected waste cost"]
values = output_full[money_cols].sum().values.round(2)
dct = {}
i = 0
for name in money_cols:
    dct[name] = values[i]
    i += 1
dct 

NameError: name 'output_full' is not defined

In [5]:
results = pd.DataFrame({"Metric":["Loss revenue", "Loss profit", "Waste cost","Loss revenue", "Loss profit", "Waste cost"], "Method":["Actual", "Actual", "Actual", "Predicted w/discounts", "Predicted w/discounts", "Predicted w/discounts"], "values":output_full[money_cols].sum().values.round(1)})
results["product"] = ["name", "name", "name", "name", "name", "name"]
results

,Metric,Method,values,product
0,Loss revenue,Actual,2484.8,name
1,Loss profit,Actual,1886.9,name
2,Waste cost,Actual,597.8,name
3,Loss revenue,Predicted w/discounts,2106.3,name
4,Loss profit,Predicted w/discounts,1564.6,name
5,Waste cost,Predicted w/discounts,541.8,name


In [31]:
import plotly.express as px
fig = px.bar(results, x="Metric", y='values', color="Method", barmode='group', text="values")
fig.update_layout(showlegend=False, xaxis_title="Product",
    yaxis_title="Out of Stock Event Count",width=1000,
    height=700, title={
                'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        paper_bgcolor='rgba(0,0,0,0)')
fig.show()

In [7]:
import plotly.express as px

output_copy = output_full.groupby("week").sum()
output_copy["MA7"] = output_copy['expected loss revenue'].rolling(7).mean()
output_copy["MA7_exp"] = output_copy["loss revenue"].rolling(7).mean()
fig = px.line(output_copy, x=output_copy.index, y="MA7_exp")
fig.add_scatter(x=output_copy.index, y=output_copy["MA7"], mode="lines")
fig.show()

In [7]:
bio_products = ['Biologische rode kool', 'Biologische pompoen',
       'Biologische cherry tomaten', 'Biologische prei',
       'Biologische Bloemkool', 'Biologische courgette',
       'Biologische brocolli', 'Biologische knoflook',
       'Biologische kikkererwten']
snacks = products[products["category"] == "meat"]["description"].values


In [8]:
money_dct = {}
list_of_df = []

for prod in snacks:
    try:
        print("Current product is {}".format(prod))
        inventory = pd.read_csv("data/inventory.csv")
        products = pd.read_csv("data/products.csv")
        promotions = pd.read_csv("data/promotions.csv")
        transactions = pd.read_csv("data/transactions.csv")
        discount_per_day = pd.read_csv("nr_discounts_per_day.csv")
        pd.set_option('mode.chained_assignment', None) #turns off pandas warnings
        #create a reference dataframe
        dayofyear = list(range(365))
        date = [datetime.datetime(2018, 1, 1) + datetime.timedelta(day) for day in dayofyear]
        df_date = pd.DataFrame({"day":dayofyear, "date":date})

        tup_list2 = [('Blauwe bessen', '150g'), ('Rundergehakt', '300g '), ('Unox Gelderse rookworst', '285g'), ('Biologisch rundergehakt', '300g')]
        for pair in tup_list2:

            transactions.loc[transactions[(transactions['description'] == pair[0]) & (transactions['size'] == pair[1])].index, 'description'] = pair[0] + '.1'

            products.loc[products[(products['description'] == pair[0]) & (products['size'] == pair[1])].index, 'description'] = pair[0] + '.1'

        #for colname in inventory.columns[2:]:
        #    inventory[colname] = [ast.literal_eval(i) for i in inventory[colname]]
            #converting inventory rows to list of tuples from string

        product = prod # item to be analyzed
        filtered_transactions = transactions[transactions["description"] == product]
        filtered_inventory = inventory[["day", "before or after delivery", product]]

        output_full = waste_analysis(inventory, transactions, products, product, discount_per_day=discount_per_day)
        list_of_df.append(output_full)
        values = output_full[money_cols].sum().values.round(2)
        print("Values: {}".format(values))
        i = 0
        for name in money_cols:
            if name not in money_dct.keys():
                money_dct[name] = values[i]
            else:
                money_dct[name] += values[i]
            i += 1
    except(ValueError):
            print(">>>> {} IS NOT INCLUDED".format(prod))


^2 score: 0.18186480872601873
Demand function fitted
Average of actual/estimated purchase ratios: 1.012732027697424
Waste change predicted, calculating revenue loss...
Values: [105.78  86.1   19.68  63.85  50.89  12.96]
Current product is Unox Gelderse rookworst.1
(102, 7)
Extended waste dataframe created
Data prepared for prediction
Model fitted; test set metrics: MAE: 1.5446772219193274 MSE: 3.4786190291799595, R^2 score: 0.006039515624617131
Demand function fitted
Average of actual/estimated purchase ratios: 1.0007507031397884
Waste change predicted, calculating revenue loss...
Values: [5542.7  4458.28 1084.42 5159.97 4121.59 1038.4 ]
Current product is Half om half gehakt
(102, 7)
Extended waste dataframe created
Data prepared for prediction
Model fitted; test set metrics: MAE: 1.7407283967514682 MSE: 5.3827441694822245, R^2 score: 0.08655886511170474
Demand function fitted
Average of actual/estimated purchase ratios: 1.0219341543200462
Waste change predicted, calculating revenue l

In [9]:
results = pd.DataFrame({"Metric":["Loss revenue", "Loss profit", "Waste cost","Loss revenue", "Loss profit", "Waste cost"], "Method":["Actual", "Actual", "Actual", "Predicted w/discounts", "Predicted w/discounts", "Predicted w/discounts"], "values":[round(i,2) for i in money_dct.values()]})
results

,Metric,Method,values
0,Loss revenue,Actual,35859.71
1,Loss profit,Actual,28167.22
2,Waste cost,Actual,7692.49
3,Loss revenue,Predicted w/discounts,31180.16
4,Loss profit,Predicted w/discounts,24362.39
5,Waste cost,Predicted w/discounts,6817.89


In [10]:
import plotly.express as px
fig = px.bar(results, x="Metric", y='values', color="Method", barmode='group', text="values", title="Savings with optimized waste management for bio Vegetables")
fig.update_layout(showlegend=False, xaxis_title="Product",
    yaxis_title="Total loss in €",width=1000,
    height=700, title={
        
                'y':0.9,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
        paper_bgcolor='rgba(0,0,0,0)')
fig.show()

In [2]:
from waste_functions import create_bb_data, add_purchases

waste_df = create_bb_data(filtered_inventory, product)
waste_df_extended = add_purchases(waste_df, transactions, product)
waste_df_extended

(101, 5)


,amount,DOY,cumulative purchases,purchases,remaining,waste,week,waste nn
best before day,,,,,,,,
14,26,14.0,42.0,NaN,-16.0,-16.0,3,0.0
17,26,17.0,52.0,10.0,16.0,0.0,3,0.0
21,26,21.0,71.0,19.0,7.0,7.0,4,7.0
24,26,24.0,77.0,6.0,20.0,20.0,4,20.0
28,26,28.0,95.0,18.0,8.0,8.0,5,8.0
...,...,...,...,...,...,...,...,...
350,26,350.0,1268.0,11.0,15.0,15.0,51,15.0
353,26,353.0,1278.0,10.0,16.0,16.0,51,16.0
357,26,357.0,1290.0,12.0,14.0,14.0,52,14.0


## Creating range intervals for mid-delivery sales calculation

In [3]:
best_before = []
for element_list in inventory[product]:
    for batch in element_list:
        best_before.append(batch[0])
bb_dates = list(np.unique(best_before))
#list of each unique best before dates
bb_dates.insert(0, 0)
ranges = []
i = 0
for item in bb_dates:
    if i < len(bb_dates)-1:
        i += 1
        ret_tupl = (item, bb_dates[i])
        ranges.append(ret_tupl)



In [4]:
ranges[4]

(24, 28)

In [30]:
from demand import *
compl_prod="Basis anannasstukjes"

prepared_transactions = prepare_data(filtered_transactions, discounts_per_day=discount_per_day)
output_dct = prepare_demand_function(prepared_transactions)

In [39]:
last = len(prepared_transactions)-1
pd.DataFrame(prepared_transactions.values, columns=prepared_transactions.columns).loc[last-1:]

,count,discount,std,min,25%,50%,75%,max,month,product,DOY,dayofweek,purchase_price,std_sales_price,prev_day_purchases,days,on_discount
346,2,0,0,0,0,0,0,0,December,Biologische knoflook,362,Saturday,0.69,0.69,4,29-12-2018,12
347,2,0,0,0,0,0,0,0,December,Biologische knoflook,363,Sunday,0.69,0.69,2,30-12-2018,13


In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
rf = RandomForestRegressor(n_estimators=50, max_depth=9)
nn =  MLPRegressor(hidden_layer_sizes=(50,50))
dtr = DecisionTreeRegressor()
model = fit_demand_function(output_dct)

Model fitted; test set metrics: MAE: 1.8313170478097898 MSE: 4.959650043989804, R^2 score: -0.14613332060051532


In [7]:
output_dct["train"][0]

,discount,purchase_price,prev_day_purchases,on_discount,x0_Friday,x0_Monday,x0_Saturday,x0_Sunday,x0_Thursday,x0_Tuesday,...,x0_December,x0_February,x0_January,x0_July,x0_June,x0_March,x0_May,x0_November,x0_October,x0_September
0,0,1.79,0,15,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,1.79,1,11,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0,1.79,1,22,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0,1.79,2,15,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0,1.79,1,14,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,0,1.79,6,15,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
260,0,1.79,2,11,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
261,0,1.79,5,24,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
262,0,1.79,4,15,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [8]:

test = predicted_demand(waste_df_extended, ranges, model, output_dct, prepared_transactions)

Value error occurred
Value error occurred
Value error occurred


In [9]:
test['purchases'].iloc[0] = test["cumulative purchases"].iloc[0]

In [10]:
waste_df_extended["expected waste"].fillna(0, inplace=True)

In [11]:
purchase_price = products[products["description"] == product]["purchase_price"].values
waste_df_extended.fillna({"expected waste":0}, inplace=True)
waste_df_extended["expected loss revenue"] = waste_df_extended["expected waste"]*waste_df_extended["avg_price"]
waste_df_extended["expected loss profit"] = waste_df_extended["expected waste"]*(waste_df_extended["avg_price"]-purchase_price)
waste_df_extended["expected waste cost"] = waste_df_extended["expected waste"]*purchase_price

waste_df_extended["loss revenue"] = waste_df_extended["waste nn"]*waste_df_extended["avg_price"]
waste_df_extended["loss profit"] = waste_df_extended["waste nn"]*(waste_df_extended["avg_price"]-purchase_price)
waste_df_extended["waste cost"] = waste_df_extended["waste nn"]*purchase_price
waste_df_extended

,amount,DOY,cumulative purchases,purchases,remaining,waste,week,waste nn,predicted demand,demand_discounts,avg_price,expected purchases,expected waste,expected loss revenue,expected loss profit,expected waste cost,loss revenue,loss profit,waste cost
best before day,,,,,,,,,,,,,,,,,,,
14,26,14.0,42.0,42.0,-16.0,-16.0,3,0.0,41,41,1.790000,NaN,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00
17,26,17.0,52.0,10.0,16.0,0.0,3,0.0,15,15,1.790000,20.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.00
21,26,21.0,71.0,19.0,7.0,7.0,4,7.0,16,33,1.491667,39.0,0.0,0.000000,0.000000,0.00,10.441667,7.571667,2.87
24,26,24.0,77.0,6.0,20.0,20.0,4,20.0,16,33,1.432000,12.0,14.0,20.048000,14.308000,5.74,28.640000,20.440000,8.20
28,26,28.0,95.0,18.0,8.0,8.0,5,8.0,16,33,1.271842,37.0,0.0,0.000000,0.000000,0.00,10.174737,6.894737,3.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350,26,350.0,1268.0,11.0,15.0,15.0,51,15.0,21,38,1.541389,22.0,4.0,6.165556,4.525556,1.64,23.120833,16.970833,6.15
353,26,353.0,1278.0,10.0,16.0,16.0,51,16.0,13,30,1.057727,20.0,6.0,6.346364,3.886364,2.46,16.923636,10.363636,6.56
357,26,357.0,1290.0,12.0,14.0,14.0,52,14.0,16,33,1.193333,25.0,1.0,1.193333,0.783333,0.41,16.706667,10.966667,5.74


In [12]:
import plotly.express as px

fig = px.line(waste_df_extended, x="DOY", y="expected loss revenue")
fig.add_scatter(x=waste_df_extended["DOY"], y=waste_df_extended["loss revenue"], mode="lines")
fig.show()

In [11]:
actual_purchases = test['purchases'].sum()
actual_purchases 

1418.0

In [12]:
pred_demand_discounts = test["demand_discounts"].sum()

In [13]:
pred_demand = test["predicted demand"].sum()
pred_demand

1806.9149483893552

In [14]:
(pred_demand_discounts/pred_demand)*actual_purchases

1734.6240243785185